In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score

import pandas as pd
import numpy as np

## Carrega as duas bases para testes

In [5]:
# Com categorização das colunas bmi e glicose
stroke_cat = pd.read_csv('stroke_df_cat')
# Sem Outliers
stroke = pd.read_csv('stroke_df_sem_out')

## Analisando o primero caso

## Entrada para modelagem

In [7]:
#features

X = (stroke_cat.drop(['stroke'], axis=1)).values

#target

y = (stroke_cat['stroke']).values

## Criação dos modelos

In [15]:
ad_clf = DecisionTreeClassifier()
knn_clf = KNeighborsClassifier(n_neighbors = 5)
rf_clf = RandomForestClassifier(n_estimators=100)
lg_clf = LogisticRegression()
mlp_clf = MLPClassifier(random_state=1, max_iter=300)

## Amostragem Holdout teste dos modelos

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33,stratify=y)

## Oversampling

In [40]:
from imblearn.over_sampling import SMOTE

s=SMOTE()
X_res,y_res =s.fit_resample(X,y)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.33)

## Undersamplig

In [47]:
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler

rus= RandomUnderSampler()
cc = ClusterCentroids()

X_res,y_res = rus.fit_resample(X,y)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.33)

## Treinar modelos

In [50]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]

# Dicionário para armazenar métricas
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

for modelo in modelos:
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    modelo_name = modelo.__class__.__name__
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.717703   0.759358  0.660465  0.706468
1    KNeighborsClassifier  0.755981   0.781095  0.730233  0.754808
2  RandomForestClassifier  0.772727   0.775229  0.786047  0.780600
3      LogisticRegression  0.787081   0.805825  0.772093  0.788599
4           MLPClassifier  0.777512   0.785047  0.781395  0.783217


## Treinar modelos com  o KFold

In [33]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]


metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

# Realize a validação cruzada e calcule as métricas para cada modelo
for modelo in modelos:
    modelo_name = modelo.__class__.__name__
    
    # Realize a validação cruzada com previsões
    y_pred = cross_val_predict(modelo, X, y, cv=5)
    
    # Calcule as métricas
    accuracy = cross_val_score(modelo, X, y, cv=5, scoring='accuracy').mean()
    precision = cross_val_score(modelo, X, y, cv=5, scoring='precision').mean()
    recall = cross_val_score(modelo, X, y, cv=5, scoring='recall').mean()
    f1 = cross_val_score(modelo, X, y, cv=5, scoring='f1').mean()
    
    # Armazene as métricas no dicionário
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas
import pandas as pd
metrics_df = pd.DataFrame(metrics)
print(metrics_df)


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.933738   0.184573  0.178678  0.185042
1    KNeighborsClassifier  0.955760   0.306128  0.060067  0.100375
2  RandomForestClassifier  0.948964   0.235116  0.093338  0.124097
3      LogisticRegression  0.958309   0.233333  0.004762  0.009327
4           MLPClassifier  0.958636   0.000000  0.000000  0.000000


In [51]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]


metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

# Realize a validação cruzada e calcule as métricas para cada modelo
for modelo in modelos:
    modelo_name = modelo.__class__.__name__
    
    # Realize a validação cruzada com previsões
    y_pred = cross_val_predict(modelo, X_res, y_res, cv=5)
    
    # Calcule as métricas
    accuracy = cross_val_score(modelo, X_res, y_res, cv=5, scoring='accuracy').mean()
    precision = cross_val_score(modelo, X_res, y_res, cv=5, scoring='precision').mean()
    recall = cross_val_score(modelo, X_res, y_res, cv=5, scoring='recall').mean()
    f1 = cross_val_score(modelo, X_res, y_res, cv=5, scoring='f1').mean()
    
    # Armazene as métricas no dicionário
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas
import pandas as pd
metrics_df = pd.DataFrame(metrics)
print(metrics_df)


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.727037   0.733681  0.720047  0.730798
1    KNeighborsClassifier  0.778477   0.787876  0.764342  0.774297
2  RandomForestClassifier  0.795094   0.779576  0.816485  0.797666
3      LogisticRegression  0.802237   0.797571  0.811836  0.803314
4           MLPClassifier  0.778474   0.759913  0.830659  0.789253
